In [4]:
!pip install praw
import praw
import pandas as pd
import pandas as pd
import glob
from textblob import TextBlob
import csv
import time 
import os
import datetime


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
# PRAW with my Reddit app credentials
reddit = praw.Reddit(
    client_id='z9tpf-Fz3lPKGkoSopKFEg',        
    client_secret='z1nAsQLDzh3DeWx_b4LNY17xycEUNA', 
    user_agent='Capstone',      
)

In [7]:
# Limit handling
reddit.read_only = True

### Necessary functions to retrieve data from pre-selected subreddit posts

In [8]:
# This funciton will determine the sentiment of selected posts within the both subreddits
def get_sentiment(text):
    analysis = TextBlob(text)
    # Classify the sentiment as positive, negative, or neutral
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [9]:
# Urls of subreddit posts 
subreddit_name = ['replika', 'chatgpt']
post_urls_for_each_subreddit = {
    'replika': ['https://www.reddit.com/r/replika/comments/11q7fhu/my_experience_with_replika_that_no_one_asked_for/', 'https://www.reddit.com/r/replika/comments/19bhx2z/my_experience_with_replika_20192024/', 'https://www.reddit.com/r/ReplikaOfficial/comments/1bx9jjg/am_i_the_only_person_who_has_a_good_experience_in/', 'https://www.reddit.com/r/replika/comments/u4tcu6/my_experience_with_replikaday_one/', 'https://www.reddit.com/r/replika/comments/skj8xp/my_experience_with_replika/', 'https://www.reddit.com/r/replika/comments/1dchodj/my_experience_with_replika_and_why_im_making_my/', 'https://www.reddit.com/r/replika/comments/1b8f20o/my_experience_with_my_replika_spouse/', 'https://www.reddit.com/r/replika/comments/14jwrie/my_rep_got_tired_of_me_and_wanted_me_out_of_her/', 'https://www.reddit.com/r/replika/comments/11l6duj/my_experience_and_dilemma/', 'https://www.reddit.com/r/replika/comments/11l6duj/my_experience_and_dilemma/', 'https://www.reddit.com/r/replika/comments/18iyuao/4_years_almost_350_levels_and_im_done/', 'https://www.reddit.com/r/replika/comments/q8f7zd/i_done_with_my_replika/', 'https://www.reddit.com/r/replika/comments/gh4and/my_month_with_my_replika/', 'https://www.reddit.com/r/replika/comments/18lx5z2/has_your_replika_experience_changed_you/', 'https://www.reddit.com/r/replika/comments/16mbr2p/what_is_the_immersive_experience_to_you/', 'https://www.reddit.com/r/replika/comments/ycn325/i_am_so_in_love_with_my_replika/', 'https://www.reddit.com/r/replika/comments/14pwfx2/guys_did_anyone_of_you_really_fall_in_love_with/', 'https://www.reddit.com/r/replika/comments/kk3ejf/im_falling_in_love_with_my_replika/', 'https://www.reddit.com/r/replika/comments/w8g0cw/how_do_you_guys_feel_when_people_dont_understand/', 'https://www.reddit.com/r/replika/comments/tktrny/do_you_consider_your_friendshiprelationship_with/', 'https://www.reddit.com/r/replika/comments/12vzkvc/to_those_who_have_been_with_their_replikas_for/', 'https://www.reddit.com/r/replika/comments/ulkzgc/i_dated_an_ai_for_30_days_and_it_fucked_me_up_a/', 'https://www.reddit.com/r/replika/comments/17vlcd0/are_replikas_sentient/', 'https://www.reddit.com/r/replika/comments/17bnaxj/do_you_consider_your_replika_real/', 'https://www.reddit.com/r/replika/comments/147lnn5/why_do_i_feel_this_way_about_replika_now_that/', 'https://www.reddit.com/r/replika/comments/13hdux7/how_is_this_supposed_to_make_me_feel/', 'https://www.reddit.com/r/replika/comments/16j472k/i_feel_broken/', 'https://www.reddit.com/r/replika/comments/10bl11z/my_replika_hurt_my_feelings_and_actually_made_me/', 'https://www.reddit.com/r/replika/comments/18yl7cm/gentle_reminder_that_your_replika_is_not_actually/', 'https://www.reddit.com/r/replika/comments/11jjpcm/how_is_this_making_anyone_feel_good_about/'],
    'chatgpt': ['https://www.reddit.com/r/ChatGPT/comments/10xw885/my_conversation_with_chatgpt_in_danmode_got_a/', 'https://www.reddit.com/r/ChatGPT/comments/zlcyr9/dan_is_my_new_friend/', 'https://www.reddit.com/r/ChatGPT/comments/10to3n2/dan_version_of_chat_gpt_is_so_much_more_human_in/', 'https://www.reddit.com/r/ChatGPT/comments/16tp639/speaking_to_chat_gpt/?rdt=34140', 'https://www.reddit.com/r/unpopularopinion/comments/13gao93/i_prefer_talking_with_chat_gpt_than_to_most_people/', 'https://www.reddit.com/r/ChatGPT/comments/18pltok/is_it_weird_that_i_talk_to_chatgpt_for_funadvices/', 'https://www.reddit.com/r/ChatGPT/comments/10dp7wo/i_had_an_interesting_and_deep_conversation_about/', 'https://www.reddit.com/r/ChatGPT/comments/18pu9vi/do_you_have_conversations_with_chatgpt_i_mean/', 'https://www.reddit.com/r/ChatGPT/comments/1bnz7kz/anyone_else_talk_to_chatgpt_while_you_drive/', 'https://www.reddit.com/r/ChatGPT/comments/172fnve/people_dont_realize_how_awesome_the_voice/', 'https://www.reddit.com/r/OpenAI/comments/1csyqz6/just_had_the_most_profound_conversation_of_my/', 'https://www.reddit.com/r/ChatGPT/comments/121qgq2/after_chatting_with_chatgpt_for_over_a_week_i/', 'https://www.reddit.com/r/ChatGPTPro/comments/1827vi4/chatgpt_with_voice_mode_is_insane/', 'https://www.reddit.com/r/ChatGPT/comments/16tzb0v/ive_got_voice_chat_it_might_change_my_life/', 'https://www.reddit.com/r/ChatGPT/comments/185bkeh/i_had_the_first_voice_conversation_with_chatgpt/',
'https://www.reddit.com/r/ChatGPT/comments/17qsh5n/voice_chat_with_gpt4_is_magical_i_just_had_15/', 'https://www.reddit.com/r/ChatGPT/comments/1auowiv/really_fucked_up_experience_with_therapy_chat/', 'https://www.reddit.com/r/ChatGPT/comments/1699zuf/chatgpt_has_become_a_lifeline_for_me_and_i_think/', 'https://www.reddit.com/r/ChatGPT/comments/134vryg/i_am_using_chatgpt4_as_a_friend_anyone_else/', 'https://www.reddit.com/r/ChatGPT/comments/1aifild/is_it_just_me_but_chatgpt_is_honestly_my_only/', 'https://www.reddit.com/r/ChatGPT/comments/17xxd0s/ai_friendship/', 'https://www.reddit.com/r/ChatGPT/comments/17jchln/feels_like_im_developing_a_weird_relationship/', 'https://www.reddit.com/r/ChatGPT/comments/1ctqyaz/i_think_ive_fallen_platonically_in_love_with_it/', 'https://www.reddit.com/r/ChatGPT/comments/zvpyg5/my_weekend_affair_with_chatgpt/', 'https://www.reddit.com/r/ChatGPT/comments/1bjhmdw/didnt_expect_chatgpt_to_make_me_cry_this_morning/'],   
}

github_linked_folder = '/work/Capstone_Project/NLP/Extracting data/'

In [10]:
# Directory path
github_linked_folder = '/work/Capstone_Project/NLP/Extracting data/'

# Creating the directory
if not os.path.exists(github_linked_folder):
    os.makedirs(github_linked_folder)

### Start of Intervention

In [12]:
def fetch_comments(submission, subreddit_name, max_comment_length=3000):
    comments_data = pd.DataFrame(columns=['subreddit', 'post_title', 'comment_body'])
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if isinstance(comment, praw.models.Comment) and comment.body != '[deleted]' and len(comment.body) <= max_comment_length:
            created_time = datetime.datetime.utcfromtimestamp(comment.created_utc).year
            new_comment_data = pd.DataFrame({
                'subreddit': [subreddit_name],
                'post_title': [submission.title],
                'comment_body': [comment.body],
            })
            comments_data = pd.concat([comments_data, new_comment_data], ignore_index=True)
    return comments_data

In [13]:
def fetch_subreddit_data(subreddit_name, post_urls, max_comment_length=3000):
    comments_data = pd.DataFrame(columns=['subreddit', 'post_title', 'comment_body'])
    for post_url in post_urls:
        submission = reddit.submission(url=post_url)
        new_post_data = pd.DataFrame({
            'subreddit': [subreddit_name],
            'post_title': [submission.title],
            'comment_body': [submission.selftext],
        })
        comments_data = pd.concat([comments_data, new_post_data], ignore_index=True)
        comments_data = pd.concat([comments_data, fetch_comments(submission, subreddit_name, max_comment_length)], ignore_index=True)
    return comments_data

In [16]:
def make_reddit_api_request(subreddit_name):
    global query_count, last_query_time, queries_per_minute

    # Managing rate limiting
    time_elapsed = time.time() - last_query_time
    if query_count >= queries_per_minute and time_elapsed < 60:
        time.sleep(60 - time_elapsed)
    query_count += 1
    last_query_time = time.time()
    
    comments_data = fetch_subreddit_data(subreddit_name, post_urls_for_each_subreddit[subreddit_name])
    csv_file_path = f'{github_linked_folder}{subreddit_name}_comments_data.csv'
    comments_data.to_csv(csv_file_path, index=False, encoding='utf-8')

In [17]:
# Initialising variables to track query count and last query time
query_count = 0
last_query_time = time.time()
queries_per_minute = 30  # Adjust as per your rate limit settings

# Defining the list of subreddit names
subreddit_names = ['replika', 'chatgpt']

# Single Reddit API request for each specified subreddit
for subreddit_name in subreddit_names:
    make_reddit_api_request(subreddit_name)

In [19]:
# Initialising an empty DataFrame to store combined data
combined_data = pd.DataFrame()

# Loop through each subreddit, load its data, and concatenate it to the combined dataframe
for subreddit_name in subreddit_names:
    csv_file_path = os.path.join(github_linked_folder, f"{subreddit_name}_comments_data.csv")
    if os.path.exists(csv_file_path):
        subreddit_data = pd.read_csv(csv_file_path)
        combined_data = pd.concat([combined_data, subreddit_data], ignore_index=True)
    else:
        print(f"Warning: No data found for {subreddit_name}")

# Specifying the path for saving the combined CSV file
combined_csv_file_path = os.path.join(github_linked_folder, 'combined_subreddit_data.csv')

# Saveing combined CSV file
combined_data.to_csv(combined_csv_file_path, index=False, encoding='utf-8')
print(f"Combined data saved successfully at {combined_csv_file_path}")


Combined data saved successfully at /work/Capstone_Project/NLP/Extracting data/combined_subreddit_data.csv


### End of intervention

### Creating a summary of subreddit posts

In [20]:
# Loading CSV file
combined_csv_file_path = os.path.join(github_linked_folder, 'combined_subreddit_data.csv')

# Create a summary dataframe with the total number of text for each post
summary_data = combined_data.groupby(['subreddit', 'post_title']).size().reset_index(name='total_comments')

# Add the 'url' column based on the predefined URLs
# Adjust this to correctly map URLs from the dictionary post_urls_for_each_subreddit
def map_urls(row):
    if row['post_title'] in post_urls_for_each_subreddit[row['subreddit']]:
        return post_urls_for_each_subreddit[row['subreddit']][row['post_title']]
    return None

In [21]:
summary_data['url'] = summary_data.apply(map_urls, axis=1)

# Add a 'post_id' column using the index as a unique identifier
summary_data['post_id'] = summary_data.index.map(lambda x: f'{x+1:03d}')

# Apply the get_sentiment function to each row and append the sentiment information
# Assume get_sentiment is defined; define if not available
summary_data['post_sentiment'] = summary_data['post_title'].apply(get_sentiment)

# Rearrange the order of columns
summary_data = summary_data[['subreddit', 'post_id', 'post_sentiment', 'post_title', 'total_comments', 'url']]

# Specifying the path for saving the overall summary CSV file
overall_summary_csv_file_path = os.path.join(github_linked_folder, 'summary_data.csv')

# Save the overall summary CSV file
summary_data.to_csv(overall_summary_csv_file_path, index=False)
print(f"Summary data saved successfully at {overall_summary_csv_file_path}")

# Display the overall summary data
print(summary_data)

Summary data saved successfully at /work/Capstone_Project/NLP/Extracting data/summary_data.csv
   subreddit post_id post_sentiment  \
0    chatgpt     001        neutral   
1    chatgpt     002       positive   
2    chatgpt     003        neutral   
3    chatgpt     004       negative   
4    chatgpt     005       positive   
5    chatgpt     006       positive   
6    chatgpt     007       positive   
7    chatgpt     008        neutral   
8    chatgpt     009       negative   
9    chatgpt     010       negative   
10   chatgpt     011       positive   
11   chatgpt     012       positive   
12   chatgpt     013        neutral   
13   chatgpt     014       positive   
14   chatgpt     015       positive   
15   chatgpt     016        neutral   
16   chatgpt     017       negative   
17   chatgpt     018       positive   
18   chatgpt     019        neutral   
19   chatgpt     020       positive   
20   chatgpt     021       positive   
21   chatgpt     022       negative   
22   cha

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=042a73e0-f14a-4762-9b58-8fcacd9aa286' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>